### Contextual Compression in Document Retrieval
#### Overview
###### This code demonstrates the implementation of contextual compression in a document retrieval system using LangChain and OpenAI's language models. The technique aims to improve the relevance and conciseness of retrieved information by compressing and extracting the most pertinent parts of documents in the context of a given query.

#### Motivation
###### Traditional document retrieval systems often return entire chunks or documents, which may contain irrelevant information. Contextual compression addresses this by intelligently extracting and compressing only the most relevant parts of retrieved documents, leading to more focused and efficient information retrieval.

#### Key Components
###### Vector store creation from a PDF document
###### Base retriever setup
###### LLM-based contextual compressor
###### Contextual compression retriever
###### Question-answering chain integrating the compressed retriever

#### Method Details
###### Document Preprocessing and Vector Store Creation
###### The PDF is processed and encoded into a vector store using a custom encode_pdf function.
###### Retriever and Compressor Setup
###### A base retriever is created from the vector store.
###### An LLM-based contextual compressor (LLMChainExtractor) is initialized using OpenAI's GPT-4 model.
###### Contextual Compression Retriever
###### The base retriever and compressor are combined into a ContextualCompressionRetriever.
###### This retriever first fetches documents using the base retriever, then applies the compressor to extract the most relevant information.
###### Question-Answering Chain
###### A RetrievalQA chain is created, integrating the compression retriever.
###### This chain uses the compressed and extracted information to generate answers to queries.

#### Benefits of this Approach
###### Improved relevance: The system returns only the most pertinent information to the query.
###### Increased efficiency: By compressing and extracting relevant parts, it reduces the amount of text the LLM needs to process.
###### Enhanced context understanding: The LLM-based compressor can understand the context of the query and extract information accordingly.
###### Flexibility: The system can be easily adapted to different types of documents and queries.

#### Conclusion
###### Contextual compression in document retrieval offers a powerful way to enhance the quality and efficiency of information retrieval systems. By intelligently extracting and compressing relevant information, it provides more focused and context-aware responses to queries. This approach has potential applications in various fields requiring efficient and accurate information retrieval from large document collections.

##### import libraries

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path since we work with notebooks
from helper_functions import *
from evaluation.evaluate_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document's path

In [ ]:
path = "../data/Understanding_Climate_Change.pdf"

### Create a vector store

In [ ]:
vector_store = encode_pdf(path)

### Create a retriever + contexual compressor + combine them

In [ ]:
# Create a retriever
retriever = vector_store.as_retriever()

# Create a contextual compressor
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
compressor = LLMChainExtractor.from_llm(llm)

# Combine the retriever with the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Create a QA chain with the compressed retriever
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = compression_retriever,
    return_source_documents=True
)

### Example usage

In [ ]:
query = "What is the main topic of the document?"
result = qa_chain.invoke({"query": query})
print(result["result"])
print("Source documents:", result["source_documents"])